In [ ]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [17]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
}


In [18]:
links_list=[]

In [27]:
for i in range(1,21):
    url = f"https://www.amazon.in/s?k=laptops&page={i}&crid=3NH9YKCK6YS2&qid=1746475075&sprefix=laptop%2Caps%2C284&xpid=hHKSIbrfB5q68&ref=sr_pg_1"
    
    r=requests.get(url,headers=headers)
    soup=BeautifulSoup(r.text,"html.parser")
    
    links = soup.find_all('a', class_="a-link-normal")
    
    for i in links:
        href = i.get('href')
        links_list.append("https://www.amazon.in" + href)

    time.sleep(1)

In [28]:
len(links_list)

8537

In [29]:
#Product name
def Product_name(soup):
    tag = soup.find('span', id='productTitle')
    return tag.text.strip() if tag else ""

In [30]:
#Price
def Price(soup):
    tag=soup.find('span',class_='a-offscreen')
    return tag.text.strip() if tag else ""

In [31]:
#Review
def Rating(soup):
    tag=soup.find('span',class_='a-icon-alt')
    return tag.text if tag else ""

In [32]:
#desc
def Description(soup):
    tag=soup.find('div',id='feature-bullets')
    return tag.text.strip() if tag else ""

In [33]:
d={'Product_name':[],'Price':[],'Rating':[],'Description':[]}

In [ ]:
for i in links_list:
    new_page=requests.get(i,headers=headers)
    new_soup=BeautifulSoup(new_page.content,"html.parser")
    
    d['Product_name'].append(Product_name(new_soup))
    d['Price'].append(Price(new_soup))
    d['Rating'].append(Rating(new_soup))
    d['Description'].append(Description(new_soup))

    time.sleep(1)

In [12]:
amazon_df = pd.DataFrame.from_dict(d)
amazon_df['Product_name'] = amazon_df['Product_name'].replace('', np.nan)
amazon_df = amazon_df.dropna(subset=['Product_name'])
amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [25]:
df=pd.read_csv(f'amazon_data.csv')
df.head()

,Product_name,Price,Rating,Description
0,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89900.0,4.5,About this item SPEED OF LIGHTNESS — MacBoo...
1,Apple 2024 MacBook Pro Laptop with M4 Pro chip...,186990.0,5.0,About this item SUPERCHARGED BY M4 PRO OR M...
2,"Lenovo IdeaPad Slim 3, Intel Core i5-12450H, 1...",48833.0,3.9,About this item Processor: Intel Core i5-12...
3,"HP Victus, AMD Ryzen 5 5600H, NVIDIA RTX 3050,...",56490.0,4.1,About this item 【6-core AMD Ryzen 5 5600H】E...
4,"Lenovo V15 AMD Ryzen 3 7320U 15.6"" (39.62cm) F...",26990.0,3.7,About this item Processor: AMD Ryzen 3 7320...


In [26]:
df.shape
df.count()

Product_name    263
Price           263
Rating          263
Description     261
dtype: int64

In [27]:
df.isnull().sum()

Product_name    0
Price           0
Rating          0
Description     2
dtype: int64

In [44]:
df.fillna({'Description':0},inplace=True)
df.isnull().sum()

Product_name    0
Price           0
Rating          0
Description     0
dtype: int64

In [45]:
df['Price'] = df['Price'].astype(str).str.replace('₹', '').str.replace(',', '').str.strip()

In [46]:
df['Rating'] = df['Rating'].astype(str).str.extract(r'(\d+\.?\d*)')
df['Rating'] = df['Rating'].astype(float)

In [ ]:
def extract_core_name(name):
    name = str(name)
    name = re.sub(r'[\(\)\[\]]', '', name)
    
    pattern = r'^(.*?\b(?:MacBook|IdeaPad|ThinkPad|Legion|Surface|Vivobook|Zenbook|XPS|Latitude|Inspiron|Pavilion|Omen|Victus|Gram|Swift|Aspire|Yoga|Predator|ROG|TUF|Galaxy Book|MateBook)[^\d\n,]*)' 
    match = re.search(pattern, name, re.IGNORECASE)
    
    if match:
        core = match.group(1)
        core = re.sub(r'(?<=\b)(\d{4})\b', r'\1', core)  
        core = re.sub(r'\s+', ' ', core).strip()
        return core
    
   
    return name.split(',')[0].split('(')[0].strip()
    
    

In [81]:
df['Product_name'] = df['Product_name'].apply(extract_core_name)

In [82]:
df['Product_name']

0          Apple MacBook Air
1        Apple MacBook Pro M
2      Lenovo IdeaPad Slim i
3            HP Victus Ryzen
4         Lenovo V15 Ryzen 3
               ...          
258        Apple MacBook Air
259      Apple MacBook Pro M
260          Dell Inspiron i
261               HP 13th i3
262             Dell Thin i3
Name: Product_name, Length: 263, dtype: object

In [ ]:
def extract_features(description):
    if not isinstance(description, str):
        return "No description available"
    
   
    features = {
        "Processor": "Unknown",
        "RAM": "Unknown",
        "Memory": "Unknown",
        "Operating System": "Unknown",
        "Storage": "Unknown",
        "Battery Life": "Unknown"
    }
    
    
    processor_pattern = r'(?:(?:[0-9]{1,2}(?:th|rd) Gen )?(?:Intel Core i[3-9](?:-[0-9A-Z]+)?|AMD Ryzen [3-9](?: [0-9A-Z]+)?|Apple M[3-4](?: Pro| Max)?|Celeron N4500|Athlon Silver|MediaTek MT8788|Snapdragon X))'
    ram_pattern = r'(\d{1,2}GB (?:DDR[4-5]|LPDDR[4-5]x?)(?:-\d{4})?)'
    memory_pattern = r'(\d{1,2}GB (?:Unified Memory|Memory)(?!\s*(?:DDR|LPDDR)))'
    os_pattern = r'(Windows 1[0-1]|macOS|Linux|Chrome OS)'
    storage_pattern = r'(\d{2,4}GB SSD|\d{2,4}GB M\.2|\d{2,4}GB NVMe|\d{1,2}TB SSD|\d{2,4}GB PCIe)'
    battery_pattern = r'(up to \d{1,2} hours)'
    
    # Processor extraction
    processor_match = re.search(processor_pattern, description, re.IGNORECASE)
    if processor_match:
        features["Processor"] = processor_match.group(0).strip()
    
    # RAM extraction
    ram_match = re.search(ram_pattern, description, re.IGNORECASE)
    if ram_match:
        features["RAM"] = ram_match.group(0).strip()
    
    # Memory extraction
    memory_match = re.search(memory_pattern, description, re.IGNORECASE)
    if memory_match:
        features["Memory"] = memory_match.group(0).strip()
    
    # Operating System extraction
    os_match = re.search(os_pattern, description, re.IGNORECASE)
    if os_match:
        features["Operating System"] = os_match.group(0).title().strip()
    
    # Storage extraction
    storage_match = re.search(storage_pattern, description, re.IGNORECASE)
    if storage_match:
        features["Storage"] = storage_match.group(0).strip()
    
    # Battery Life extraction
    battery_match = re.search(battery_pattern, description, re.IGNORECASE)
    if battery_match:
        features["Battery Life"] = battery_match.group(0).strip()
    
    # Format the output
    formatted = (f"Processor: {features['Processor']} | RAM: {features['RAM']} | "
                 f"Memory: {features['Memory']} | Operating System: {features['Operating System']} | "
                 f"Storage: {features['Storage']} | Battery Life: {features['Battery Life']}")
    return formatted


In [84]:
df['Description'] = df['Description'].apply(extract_features)

In [91]:
df.to_csv("amazon_data.csv", index=False)
